# Cleaning Data Phase
-----------

In [1]:
import pandas as pd 
sales_df = pd.read_csv('sales.csv')
customer_df = pd.read_excel('customer.xlsx')


## Looking at data 

In [2]:
customer_df.head()

,customers_id,gender,age
0,CUST001,Male,34
1,CUST002,Female,26
2,CUST003,Male,50
3,CUST004,Male,37
4,CUST005,Male,30


In [3]:
sales_df.head()

,transaction_id,date,customer_id,category,units,sales
0,562S2779I14713HB,2/01/2023,CUST522,Beauty,6,24
1,316Y4029X49673UM,2/01/2023,CUST180,Clothing,5,60
2,302L3674R67861RZ,2/01/2023,CUST303,Electronics,1,48
3,638S2782Q44402JD,5/01/2023,CUST610,Beauty,3,9
4,637D2741D89311LY,5/01/2023,CUST367,Electronics,1,39


In [4]:
customer_df.describe()

,age
count,1000.000000
mean,41.463000
std,14.285402
min,0.000000
25%,29.000000
50%,42.000000
75%,53.000000
max,165.000000


In [5]:
sales_df.describe()

,units,sales
count,1026.000000,1026.000000
mean,4.850877,226.075049
std,3.466943,596.613696
min,1.000000,3.000000
25%,2.000000,42.000000
50%,4.000000,90.000000
75%,6.000000,162.000000
max,21.000000,5529.000000


In [6]:
customer_df.dtypes

customers_id    object
gender          object
age              int64
dtype: object

In [7]:
sales_df.dtypes

transaction_id    object
date              object
customer_id       object
category          object
units              int64
sales              int64
dtype: object

## Clean Sales Data

#### General Checks 

In [8]:
sales_df = sales_df.dropna()  # Remove rows with any missing values

# Check for duplicate transactions
duplicate_ids = sales_df[sales_df.duplicated(subset='transaction_id', keep=False)]
duplicate_ids


,transaction_id,date,customer_id,category,units,sales
917,475Z1559L69748FV,5/12/2023,CUST997,Beauty,4,52
918,475Z1559L69748FV,5/12/2023,CUST997,Beauty,1,18


Assuming these duplicate transactions represent different transactions and that customers making more than one 
transaction may buy less per sale, the best option is to aggregate these sales to sum units & sales.

In [9]:
# Aggregating transactions
sales_df = sales_df.groupby('transaction_id', as_index=False).agg({
    'date': 'first',
    'customer_id': 'first',
    'category': 'first',
    'units': 'sum',
    'sales': 'sum'
})

# All duplicates are resolved. 
sales_df[sales_df.duplicated(subset='transaction_id', keep=False)].count().transaction_id

0

#### Cleaning fields

In [10]:
   
def clean_dates(df):
    
    date_column = 'date'
    # Convert the date column to datetime
    df[date_column] = pd.to_datetime(df[date_column], dayfirst=True, errors='coerce')
    
    # Drop rows with invalid dates
    df.dropna(subset=[date_column], inplace=True)
    
    # Filter out dates outside the desired range (1995 to present)
    min_date = pd.Timestamp('01-01-1995')
    max_date = pd.Timestamp.now()
    df = df[(df[date_column] >= min_date) & (df[date_column] <= max_date)]
    
    return df
    

In [11]:
def clean_category(df, categories, corrections=None):
    """
    Cleans known misspelled categories. 
    Suggests misspelled categories for uncorrected categories.
    
    args:
        categories: list of valid categories
        corrections: optional dict, mapping misspelled categories to correct categories.
    """
    
    # Correct misspelled categories
    if corrections:
        df['category'].replace(corrections, inplace=True)
    
    
    
    # Confirm categories align with the original list
    if sorted(df['category'].unique()) == sorted(categories):
        print("Categories Align with Category List.")
    else:
        # Check for misspelled categories
        misspelled = df[~df['category'].isin(categories)]
        print("Misspelled Categories:")
        print(misspelled)
        
    df['category'] = df['category'].astype('category')

    return df


In [12]:
def clean_units(df):
    # Remove rows with units <= 0
    df = df[df['units'] > 0]

    # Ensure units are discrete (round to the nearest integer)
    df['units'] = df['units'].round().astype(int)
    return df


def clean_sales(df):
    # Remove rows with sales <= 0
    df = df[df['sales'] > 0]

    return df


In [13]:

def clean_sales_table(df):
    """
    Returns cleaned dataframe by iterating over cleaning functions.
    """
    for clean_func in cleaning_functions:
        df = clean_func(df)
    return df
    
    
# Corrections for categories
corrections = {'Be4uty': 'Beauty', 'Electronic': 'Electronics'}

cleaning_functions = [clean_dates, 
                      lambda x: clean_category(x, ['Electronics', 'Beauty', 'Clothing'], corrections),
                      clean_units,
                      clean_sales]

sales_df = clean_sales_table(sales_df)

print(sales_df)
print(sales_df.dtypes)


Categories Align with Category List.
        transaction_id       date customer_id     category  units  sales
0     100S5713W16321BY 2023-08-18     CUST878     Clothing      3     63
1     100U7280H90017IB 2023-09-21     CUST563     Clothing      7    126
2     100X9564D67104YE 2023-10-02     CUST154  Electronics      4    192
3     100Y9513C90580VX 2023-04-09     CUST278     Clothing      4    104
4     102I4872M87046QG 2023-08-05     CUST887     Clothing      4     72
...                ...        ...         ...          ...    ...    ...
1020  996Q6275M29535AL 2023-11-03     CUST181  Electronics      4     96
1021  998I9676Q63781FY 2023-01-23     CUST049  Electronics      6    150
1022  998K1605I95937EC 2023-02-08     CUST284     Clothing      7    112
1023  998K7968J56499ON 2023-10-09     CUST201  Electronics      1     23
1024  998L2365H91680ZU 2023-06-05     CUST902       Beauty      7    161

[1022 rows x 6 columns]
transaction_id            object
date              datetime64[

## Clean Customer Data
#### General Checks 

In [14]:
# Join columns should share the same name
customer_df.rename(columns={'customers_id':'customer_id'}, inplace=True)

customer_df = customer_df.dropna()  # Remove rows with any missing values

# Check for duplicate customers
duplicate_ids = customer_df[customer_df.duplicated(subset='customer_id', keep=False)]
duplicate_ids



,customer_id,gender,age


#### Cleaning fields

In [15]:
def clean_gender(df):
    # Restrict to only Male & Female
    
    accepted_values = ['Male', 'Female','M', 'F']
    
    df['gender'] = df['gender'].str.title()

    # values not accepted will be set to NA
    df.loc[~df['gender'].isin(accepted_values), 'gender'] = pd.NA

    # NAs dropped
    df = df.dropna(subset=['gender']) 
   
    df['gender'] = df['gender'].astype('category')
    
    return df
    

In [16]:
def clean_age(df):    
    #  age range: 18 - 120
    df = df[(df['age'] >= 18) & (df['age'] <= 120)]

    # replace missing values with median
    df.loc[df['age'].isna(), 'age'] = df['age'].median()

    return df

In [17]:

def clean_customer_table(df):
    """
    Returns cleaned dataframe by iterating over cleaning functions.
    """
    for clean_func in cleaning_functions:
        df = clean_func(df)
    return df


cleaning_functions = [clean_gender, clean_age]
customer_df = clean_customer_table(customer_df)

print(customer_df)
print(customer_df.dtypes)
print(customer_df.describe())

    customer_id  gender  age
0       CUST001    Male   34
1       CUST002  Female   26
2       CUST003    Male   50
3       CUST004    Male   37
4       CUST005    Male   30
..          ...     ...  ...
994     CUST995  Female   41
995     CUST996    Male   62
996     CUST997    Male   52
997     CUST998  Female   23
999    CUST1000    Male   47

[996 rows x 3 columns]
customer_id      object
gender         category
age               int64
dtype: object
              age
count  996.000000
mean    41.404618
std     13.691282
min     18.000000
25%     29.000000
50%     42.000000
75%     53.000000
max     64.000000


## Merging tables

In [18]:
# Checking for duplicate sales by customers
duplicate_customers = sales_df[sales_df.duplicated(subset=['customer_id'], keep=False)]
print("Duplicate sales by customers:")
print(duplicate_customers)

Duplicate sales by customers:
Empty DataFrame
Columns: [transaction_id, date, customer_id, category, units, sales]
Index: []


As there are no duplicate sales by customers (other than the previously aggregated), we can assume that there is no repeat customer purchases.
Therefore, the assumption of independence will not be violated as we merge these tables. 

In [19]:

def clean_customer_id(df):
    # cleaning so I can sort by customer_id 
    
    # Remove CUST from each ID
    df['customer_id'] = df['customer_id'].str.replace('CUST', '')

    df['customer_id'] = pd.to_numeric(df['customer_id'], errors='coerce')

    
    return df 



In [20]:

# inner join is default
df = pd.merge(sales_df, customer_df, on='customer_id')

# Reorder columns and drop customer id 
df = df[["transaction_id", "sales", "date", "category", "units",  "gender", "age"]]

print(df.dtypes)
df.describe()



transaction_id            object
sales                      int64
date              datetime64[ns]
category                category
units                      int32
gender                  category
age                        int64
dtype: object


,sales,date,units,age
count,982.000000,982,982.000000,982.000000
mean,200.336049,2023-07-09 17:05:00.610998016,4.681263,41.401222
min,3.000000,2023-01-02 00:00:00,1.000000,18.000000
25%,40.000000,2023-04-24 12:00:00,2.000000,29.250000
50%,90.000000,2023-07-19 00:00:00,4.000000,42.000000
75%,158.250000,2023-10-04 00:00:00,6.000000,53.000000
max,5529.000000,2024-01-01 00:00:00,21.000000,64.000000
std,523.742816,NaN,3.154408,13.707514


In [21]:
df

,transaction_id,sales,date,category,units,gender,age
0,100S5713W16321BY,63,2023-08-18,Clothing,3,Female,20
1,100U7280H90017IB,126,2023-09-21,Clothing,7,Male,20
2,100X9564D67104YE,192,2023-10-02,Electronics,4,Male,51
3,100Y9513C90580VX,104,2023-04-09,Clothing,4,Female,37
4,102I4872M87046QG,72,2023-08-05,Clothing,4,Male,59
...,...,...,...,...,...,...,...
977,996Q6275M29535AL,96,2023-11-03,Electronics,4,Male,19
978,998I9676Q63781FY,150,2023-01-23,Electronics,6,Female,54
979,998K1605I95937EC,112,2023-02-08,Clothing,7,Male,43
980,998K7968J56499ON,23,2023-10-09,Electronics,1,Male,56


In [22]:

# df.to_csv('cleaned.csv', index=False)

# serialize df 
df.to_pickle("data.pkl")